In [1]:
import time
import numpy as np
import sys
sys.path.append("..")
from utils import * 
from vis_utils import * 

2023-08-28 09:56:40.876999: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-28 09:56:41.611770: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


using concept features


In [2]:
ckpt = torch.load(checkpoint_path, map_location=gpu)
state_dict = ckpt['state_dict']
state_dict = get_regular_ckpt_from_lightning_checkpoint(state_dict)
model.load_state_dict(state_dict)
model.eval()
model = model.to(gpu)

In [2]:
num_iterations = 100  # Number of prediction iterations

times = []
xs = [50, 100, 150, 200, 240]
for x in xs:
    times = []
    for j, batch in enumerate(dataloader_comma):
        image_array,  vego, angle, distance, g, s, l = batch
        img, angle, distance, vego = image_array.to(gpu)[0:x], angle.to(gpu)[0:x], distance.to(gpu)[0:x], vego.to(gpu)[0:x]
        start_time = time.time()
        (logits, attns), concepts = model(img, angle, distance, vego)
        end_time = time.time()
        elapsed_time = end_time - start_time
        times.append(elapsed_time)
        #angle, distance, vego, logits, concepts = angle.to("cpu"), distance.to("cpu"), vego.to("cpu"), logits.detach().cpu().to("cpu"), concepts.detach().cpu().to("cpu")

    throughput = np.mean(times)
    print(f"Avg latency for {x}: {throughput:.2f} pps")

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:3! (when checking argument for argument tensors in method wrapper_cat)

In [6]:
start_time = time.time()
num_predictions = 0
max_time = 1  # Time period in seconds

tpss = []
for i in range(100):
    while time.time() - start_time < max_time:
        image_array,  vego, angle, distance, g, s, l = next(iter(dataloader_comma))
        img, angle, distance, vego = image_array.to(gpu), angle.to(gpu), distance.to(gpu), vego.to(gpu)
        (logits, attns), concepts = model(img, angle, distance, vego)
        num_predictions += 1

    throughput = num_predictions / max_time
    tpss.append(throughput)
np.mean(tpss)

1.0

In [7]:
start_time = time.time()
num_predictions = 0
max_time = 1  # Time period in seconds
tpss = []
for i in range(100):
    while time.time() - start_time < max_time:
        _, image_array,  vego, angle, distance, _, _, _, _, _ = next(iter(dataloader_nuscenes))
        img, angle, distance, vego = image_array.to(gpu), angle.to(gpu), distance.to(gpu), vego.to(gpu)
        (logits, attns), concepts = model(img, angle, distance, vego)
        num_predictions += 1

    throughput = num_predictions / max_time
    tpss.append(throughput)
np.mean(tpss)

2.0

In [9]:
image_array,  vego, angle, distance, g, s, l = next(iter(dataloader_comma))
img, angle, distance, vego = img.to(gpu), angle.to(gpu), distance.to(gpu), vego.to(gpu)
start_time = time.time()
(logits, attns), concepts = model(img, angle, distance, vego)
end_time = time.time()
latency = (end_time - start_time) * 1000  # Convert to milliseconds
print(f"Prediction latency: {latency:.2f} ms")

Prediction latency: 246.97 ms


In [19]:
p = '/home/jessica/personalized_driving_toyota/scenarios/scenarios_curated_270.txt'
with open(p) as file:
    lines = [line.strip() for line in file]
scenarios = lines

In [20]:
p = '/home/jessica/personalized_driving_toyota/scenarios/scenarios_nuscenes.txt'

In [21]:
with open(p) as file:
    lines = [line.strip() for line in file]
scenarios2 = lines

In [22]:
m = [elem not in scenarios for elem in scenarios2]

In [30]:
pd.DataFrame(np.array(scenarios2)[m]).sample(n=400).to_csv("scen2.csv", index=False, header=False)

In [ ]:
args1 = {'model_name_or_path': '/data4/jessica/data/toyota/baselines/bert-base-uncased/', 
        'config_name': '', 'tokenizer_name': '', 'num_hidden_layers': -1, 
        'hidden_size': -1, 'num_attention_heads': -1, 'intermediate_size': -1, 
        'img_feature_dim': 512, 'load_partial_weights': False, 'freeze_embedding': False, 
        'drop_out': 0.1, 'max_seq_length': 30, 'max_seq_a_length': 30, 'max_img_seq_length': 196, 
        'do_lower_case': True, 'add_od_labels': False, 'od_label_conf': 0.0, 'use_asr': False, 
        'use_sep_cap': False, 'use_swap_cap': False, 'use_car_sensor': False, 'multitask': False, 
        'only_signal': True, 'signal_types': ['course'], 'unique_labels_on': False, 'no_sort_by_conf': False,
        'mask_prob': 0.5, 'max_masked_tokens': 45, 'attn_mask_type': 'seq2seq', 'text_mask_type': 'random', 
        'tag_to_mask': ['noun', 'verb'], 'mask_tag_prob': -1, 'random_mask_prob': 0, 
        'on_memory': False, 'effective_batch_size': 1, 'per_gpu_train_batch_size': 1, 'num_workers': 10, 'limited_samples': -1, 'learning_rate': 0.0003,
         'weight_decay': 0.05, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'warmup_ratio': 0.1, 'scheduler': 'warmup_linear', 'gradient_accumulation_steps': 1, 
         'num_train_epochs': 1, 'logging_steps': 20, 'save_steps': 2000, 'restore_ratio': -1, 'device': f"cuda:{args.gpu_num}", 'seed': 88, 'local_rank': 0,
          'mixed_precision_method': 'deepspeed', 'zero_opt_stage': -1, 'amp_opt_level': 0, 'deepspeed_fp16': True, 'fairscale_fp16': False, 'pretrained_checkpoint': '', 
          'debug': False, 'debug_speed': False, 'config': 'src/configs/VidSwinBert/BDDX_multi_default.json', 'eval_model_dir': '',
        'do_train': True, 'do_test': False, 'do_eval': False, 'do_signal_eval': False, 'evaluate_during_training': True, 
          'per_gpu_eval_batch_size': 1, 'mask_img_feat': False, 'max_masked_img_tokens': 10, 'tie_weights': False, 'label_smoothing': 0, 'drop_worst_ratio': 0, 
          'drop_worst_after': 0, 'max_gen_length': 15, 'output_hidden_states': False, 'num_return_sequences': 1, 'num_beams': 1, 'num_keep_best': 1, 'temperature': 1, 
          'top_k': 0, 'top_p': 1, 'repetition_penalty': 1, 'length_penalty': 1, 'use_cbs': False, 'min_constraints_to_satisfy': 2, 'use_hypo': False,
           'decoding_constraint': False, 'remove_bad_endings': False, 'scst': False, 'sc_train_sample_n': 5, 'sc_baseline_type': 'greedy', 'cider_cached_tokens': 
           'coco_caption/gt/coco-train-words.p', 'max_num_frames': 8, 'img_res': 224, 'patch_size': 32, 'grid_feat': True, 'kinetics': '600', 'pretrained_2d': False, 
           'vidswin_size': 'base', 'freeze_backbone': False, 'use_checkpoint': True, 'backbone_coef_lr': 0.05, 'reload_pretrained_swin': False, 'learn_mask_enabled': False, 
           'loss_sparse_w': 0, 'loss_sensor_w': 0, 'sparse_mask_soft2hard': False, 'transfer_method': -1, 'att_mask_expansion': -1, 'resume_checkpoint': 'None', 
           'use_clip_model': True, 'num_gpus': 4, 'distributed': False}

swin_model = get_swin_model(args1)
bert_model, config, tokenizer = get_bert_model(args1)